In [1]:
import sys
sys.path.append("..")
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torch.optim import SGD
import asyncio
from moxi import assemble_network

def create_synthetic_data(num_nodes=5, samples_per_node=100, input_dim=10, noise=0.1):
    """
    Creates synthetic regression data split across multiple nodes
    
    Args:
        num_nodes: Number of nodes in the network
        samples_per_node: Number of samples per node
        input_dim: Dimension of input features
        noise: Standard deviation of Gaussian noise
    
    Returns:
        list of (train_loader, val_loader) pairs for each node
    """
    # Create true parameters
    true_weights = torch.randn(input_dim, 1)
    true_bias = torch.randn(1)
    
    node_datasets = []
    
    for i in range(num_nodes):
        # Generate features
        X = torch.randn(samples_per_node, input_dim)
        
        # Generate targets with noise
        y = torch.mm(X, true_weights) + true_bias + torch.randn(samples_per_node, 1) * noise
        
        # Split into train/val (80/20)
        train_size = int(0.8 * samples_per_node)
        
        X_train, y_train = X[:train_size], y[:train_size]
        X_val, y_val = X[train_size:], y[train_size:]
        
        # Create datasets
        train_ds = TensorDataset(X_train, y_train)
        val_ds = TensorDataset(X_val, y_val)
        
        # Create dataloaders
        train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=16)
        
        node_datasets.append((train_loader, val_loader))
    
    return node_datasets

# Create the data
data = create_synthetic_data(num_nodes=5, samples_per_node=100, input_dim=10)


In [2]:
# Simple regression model to use with the data
class SimpleRegression(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, 1)
        
    def forward(self, x):
        return self.linear(x)

model = SimpleRegression(input_dim=10)


In [ ]:
global_conf = {
  "network_name": "federated_mnist_experiment",
  "network_type": "decentralised",
  "model_type": "parametric",
  "ml_framework": "pytorch",
  "comms": "async",
  "federated_rounds": 5,
  "network_size": 5,
  "metrics": ["mean_perfomance","convergence"],
  "logger": "mlflow",
  "node_base_config":{
      "model":None,
      "learning_rate": 1e-3,
      "optimizer": SGD,
      "train_data":None,
      "val_data":None,
      "criterion":nn.MSELoss,
      "random_sampling":False,
      "n_epochs": 5
  },
  "adjcency_matrix": None,
  "model": None }

In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Make sure execute_run is async
async def execute_run():
    flnw = assemble_network(global_conf, model, data)
    loss_val = await flnw.train(num_rounds=10, epochs_per_round=5)  # FL train function must be async
    return loss_val

# In Jupyter, just await instead of asyncio.run
loss_history = await execute_run()


2025/09/08 00:57:55 INFO mlflow.tracking.fluent: Experiment with name 'Federated_Learning' does not exist. Creating a new experiment.


In [5]:
loss_history

defaultdict(list, {})